In [8]:
import time

import pdb
import pprint
import nsepython
import nsetools
from nsepython import nse_get_index_list,nse_get_index_quote
import pandas as pd
#from nselib import capital_marke
from NseUtility import Nseutils, NseUtils
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', None)          # Don't break into multiple lines
pd.set_option('display.expand_frame_repr', False)  # Disable wrapping

In [ ]:
Dataaa = Nseutils.NseUtils.get_52week_high()

In [ ]:
from nsepython import nse_get_index_list

indices = nse_get_index_list()

print(indices[:5])

In [ ]:
from nsepython import nse_get_index_quote
T = nse_get_index_quote("NIFTY PSU BANK")
print(T)

In [ ]:
indices

In [ ]:
for idx in indices:
	print(nse_get_index_quote(idx))
	time.sleep(1)


In [ ]:
nse_get_index_quote("NIFTY MS IT TELCM")

In [ ]:
nsepython.fnolist()

In [ ]:
for idx in indices:
	q = nse_get_index_quote(idx)
	eq= q.get("indexType")
	sc = q.get("indexSubType")
	#print(eq)
	#pdb.set_trace()
	if (eq == "eq" and sc == "sc"):
	    print(q.get("indexName"))



In [ ]:
eq_indices = []

In [ ]:
nsepython.nse_fno("COFORGE")

In [ ]:
from nsepython import nse_get_index_quote
import pandas as pd

sector_data = []
for sector in eq_indices:
    q = nse_get_index_quote(sector)
    change_pct = q.get("percentChange") or q.get("pChange") or 0.0

    sector_data.append({
        "Sector": sector,
        "LTP": q.get("last"),
        "Prev Close": q.get("previousClose"),
        "Change %": change_pct
    })

df = pd.DataFrame(sector_data).sort_values("Change %", ascending=False)
print(df.head(10))   # top 10 trending sectors


In [ ]:
import pandas as pd
import requests
from io import StringIO
from urllib.parse import quote
from time import sleep

SECTORAL_INDICES = [
    "NIFTY AUTO",
    "NIFTY BANK",
    "NIFTY CHEMICALS",
    "NIFTY CONSUMER DURABLES",
    "NIFTY FINANCIAL SERVICES",
    "NIFTY FINANCIAL SERVICES 25/50",
    "NIFTY FINANCIAL SERVICES EX-BANK",
    "NIFTY FMCG",
    "NIFTY HEALTHCARE",
    "NIFTY IT",
    "NIFTY MEDIA",
    "NIFTY METAL",
    "NIFTY PHARMA",
    "NIFTY PRIVATE BANK",
    "NIFTY PSU BANK",
    "NIFTY REALTY",
    "NIFTY OIL & GAS",
    # MidSmall sectoral variants appear on NSE; include if needed:
    "NIFTY500 HEALTHCARE",
    "NIFTY MIDSMALL FINANCIAL SERVICES",
    "NIFTY MIDSMALL HEALTHCARE",
    "NIFTY MIDSMALL IT & TELECOM",
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/market-data/live-market-indices",
    "Connection": "keep-alive",
}

def csv_url_for_index(index_name: str) -> str:
    """
    NSE index constituent CSVs usually follow the pattern:
    https://nsearchives.nseindia.com/content/indices/ind_<lowercased_no_spaces_or_custom>.csv
    For many indices, replacing spaces with nothing and lowercasing works, while some need custom mapping.
    """
    # Known direct CSVs for common indices:
    known = {
        "NIFTY 50": "https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv",
        "NIFTY BANK": "https://nsearchives.nseindia.com/content/indices/ind_niftybanklist.csv",
        "NIFTY AUTO": "https://nsearchives.nseindia.com/content/indices/ind_niftyautolist.csv",
        "NIFTY IT": "https://nsearchives.nseindia.com/content/indices/ind_niftyitlist.csv",
        "NIFTY FMCG": "https://nsearchives.nseindia.com/content/indices/ind_niftyfmcglist.csv",
        "NIFTY PHARMA": "https://nsearchives.nseindia.com/content/indices/ind_niftypharmalist.csv",
        "NIFTY METAL": "https://nsearchives.nseindia.com/content/indices/ind_niftymetallist.csv",
        "NIFTY REALTY": "https://nsearchives.nseindia.com/content/indices/ind_niftyrealtylist.csv",
        "NIFTY MEDIA": "https://nsearchives.nseindia.com/content/indices/ind_niftymedialist.csv",
        "NIFTY PSU BANK": "https://nsearchives.nseindia.com/content/indices/ind_niftypsubanklist.csv",
        "NIFTY PRIVATE BANK": "https://nsearchives.nseindia.com/content/indices/ind_niftyprbanklist.csv",
        "NIFTY CONSUMER DURABLES": "https://nsearchives.nseindia.com/content/indices/ind_niftyconsdurableslist.csv",
        "NIFTY OIL & GAS": "https://nsearchives.nseindia.com/content/indices/ind_niftyoilgaslist.csv",
        "NIFTY HEALTHCARE": "https://nsearchives.nseindia.com/content/indices/ind_niftyhealthcarelist.csv",
        "NIFTY CHEMICALS": "https://nsearchives.nseindia.com/content/indices/ind_niftychemicalslist.csv",
        "NIFTY FINANCIAL SERVICES": "https://nsearchives.nseindia.com/content/indices/ind_niftyfinancelist.csv",
        "NIFTY FINANCIAL SERVICES 25/50": "https://nsearchives.nseindia.com/content/indices/ind_niftyfinance25_50list.csv",
        "NIFTY FINANCIAL SERVICES EX-BANK": "https://nsearchives.nseindia.com/content/indices/ind_niftyfinanceexbanklist.csv",
        # MidSmall variants may not always have public CSV; handle per-need.
    }
    if index_name in known:
        return known[index_name]
    # Fallback guess (often not reliable for rarer indices; adjust mapping above as needed)
    base = index_name.lower().replace("&", "and").replace(" ", "")
    return f"https://nsearchives.nseindia.com/content/indices/ind_{base}list.csv"

def fetch_index_constituents(session: requests.Session, index_name: str) -> pd.DataFrame:
    url = csv_url_for_index(index_name)
    for attempt in range(3):
        resp = session.get(url, timeout=15)
        if resp.status_code == 200 and resp.content:
            df = pd.read_csv(StringIO(resp.content.decode("utf-8")))
            # Normalize common columns
            for col in ["Symbol", "Company Name", "Industry", "ISIN Code", "Series"]:
                if col not in df.columns:
                    continue
            df["Index"] = index_name
            return df
        # Light backoff; 404 for non-existent CSV, 403/401 if cookies not set
        sleep(1.5 * (attempt + 1))
    raise RuntimeError(f"Failed to fetch CSV for {index_name}: HTTP {getattr(resp,'status_code', 'NA')}")

def fetch_all_sectors() -> dict:
    with requests.Session() as s:
        s.headers.update(HEADERS)
        # Warm-up to set Akamai cookies and avoid 401
        s.get("https://www.nseindia.com", timeout=8)
        result = {}
        for idx in SECTORAL_INDICES:
            try:
                df = fetch_index_constituents(s, idx)
                result[idx] = df
            except Exception as e:
                # Keep going; some indices may not expose public CSV
                result[idx] = None
        return result

if __name__ == "__main__":
    sector_constituents = fetch_all_sectors()
    # Example: print symbols per sector
    for sector, df in sector_constituents.items():
        if df is not None and "Symbol" in df.columns:
            print(sector, "=>", sorted(df["Symbol"].dropna().unique().tolist()))
        else:
            print(sector, "=>", "No CSV or unavailable")


In [ ]:
for idx in indices:
	v= nsepython.nse_get_index_quote(indices)
	v1 = v.get("'indexType': 'eq'" & "'indexSubType': 'sc'")
	print(v1.get("indexName"))

In [ ]:
nsepython.nse_get_top_gainers()

In [ ]:
nsepython.nse_get_top_losers()

In [ ]:
nsepython.get_bhavcopy(date="22-09-2025")

In [ ]:
z = nsepython.nse_get_top_losers()

In [ ]:
z

In [ ]:
from nsepython import nse_get_index_quote
import pandas as pd
from datetime import datetime

# Check current time and market status
now = datetime.now()
print(f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Note: NSE market hours are 9:15 AM - 3:30 PM IST\n")

SECTORAL_INDICES = [
    "NIFTY BANK",
    "NIFTY AUTO",
    "NIFTY FIN SERVICE",
    "NIFTY FMCG",
    "NIFTY IT",
    "NIFTY MEDIA",
    "NIFTY METAL",
    "NIFTY PHARMA",
    "NIFTY PSU BANK",
    "NIFTY PVT BANK",
    "NIFTY REALTY",
    "NIFTY HEALTHCARE",
    "NIFTY OIL AND GAS",
    "NIFTY COMMODITIES",
    "NIFTY ENERGY",
    "NIFTY INFRA",
    "NIFTY MOBILITY",
    "NIFTY IND DIGITAL"]

def safe_float_convert(value, default=0.0):
    if value is None or value == "" or value == "-":
        return default
    try:
        return float(str(value).replace(",", ""))
    except (ValueError, AttributeError):
        return default

sector_data = []

for sector in SECTORAL_INDICES:
    try:
        q = nse_get_index_quote(sector)
        
        if q is None:
            continue
        
        change_pct = safe_float_convert(q.get("percChange"))
        ltp = safe_float_convert(q.get("last"))
        prev_close = safe_float_convert(q.get("previousClose"))
        open_price = safe_float_convert(q.get("open"))
        high_price = safe_float_convert(q.get("high"))
        low_price = safe_float_convert(q.get("low"))
        
        # Get timestamp if available
        timestamp = q.get("timeVal", "N/A")
        
        if ltp == 0 and prev_close == 0:
            continue
        
        sector_data.append({
            "Sector": q.get("indexName", sector),
            "LTP": ltp,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Prev Close": prev_close,
            "Change": ltp - prev_close,
            "Change %": change_pct,
            "Updated": timestamp
        })
    except Exception as e:
        pass

df = pd.DataFrame(sector_data).sort_values("Change %", ascending=False)

print("\n=== REAL-TIME SECTORAL PERFORMANCE ===")
print(df.to_string(index=False))

print(f"\n=== MARKET SUMMARY ===")
print(f"Total sectors: {len(df)}")
print(f"Gainers: {len(df[df['Change %'] > 0])}")
print(f"Losers: {len(df[df['Change %'] < 0])}")

# Save with timestamp
filename = f'nse_sectoral_{now.strftime("%Y%m%d_%H%M%S")}.csv'
df.to_csv(filename, index=False)
print(f"\nData saved to '{filename}'")

In [ ]:
Dataa = nsepython.nse_get_index_quote("NIFTY PSU BANK")

In [ ]:
Dataa = nsepython.nse_get_advances_declines()

In [ ]:
Dataa

In [ ]:
C  = NseUtils.get_active_contracts(symbol = "TCS")

In [3]:
c = NseUtils()

In [ ]:
d = c.get_fno_full_list()

In [ ]:
d

In [13]:
d = c.get_index_details("NIFTY AUTO",list_only=True)

In [ ]:
e = c.get_gainers_losers()

In [ ]:
f = c.get_equity_full_list()

In [ ]:
g = c.rotate_user_agent()

In [14]:
d

['ASHOKLEY',
 'BAJAJ-AUTO',
 'BALKRISIND',
 'BHARATFORG',
 'BOSCHLTD',
 'EICHERMOT',
 'EXIDEIND',
 'HEROMOTOCO',
 'M&M',
 'MARUTI',
 'MOTHERSON',
 'MRF',
 'NIFTY AUTO',
 'TATAMOTORS',
 'TIINDIA',
 'TVSMOTOR']

In [6]:
Watchlist = ['Nifty India Consumption','Nifty Financial Services','Nifty Pharma','Nifty Auto','Nifty Private Bank','Nifty Commodities','Nifty MNC','Nifty Bank','Nifty Metal','Nifty Services Sector','Nifty PSE','Nifty Infrastructure','Nifty Smallcap 100','Nifty Tata Group 25% Cap','Nifty Midcap 50','Nifty PSU Bank','Nifty Energy','Nifty IT','Nifty FMCG','Nifty CPSE','Nifty50 USD','Nifty Oil & Gas','Nifty Next 50','Nifty 50','Nifty Growth Sectors 15','Nifty 100','Nifty 500','Nifty Midcap 100','Nifty Financial Services 25/50','Nifty Realty','Nifty Healthcare Index','Nifty Consumer Durables']

In [ ]:
Watchlist

In [ ]:
for idx in Watchlist:
    Out = c.get_index_details(idx)
    print(Out)
    

In [ ]:
with open("Data.txt", "a") as f:   # use "w" if you want overwrite instead of append
    for idx in Watchlist:
        Out = c.get_index_details(idx)
        f.write(f"=== {idx} ===\n")  
        f.write(Out.to_string())    # nicely formatted table
        f.write("\n\n")  
        print(Out)


In [12]:
all_data = []
failed_indices = []

print(f"Fetching data for {len(Watchlist)} indices...\n")

for i, idx in enumerate(Watchlist, 1):
    try:
        out = c.get_index_details(idx,list_only=True)
        out['index_name'] = idx
        out['fetch_date'] = datetime.now().strftime('%Y-%m-%d')
        
        all_data.append(out)
        print(f"[{i}/{len(Watchlist)}] ✓ {idx}")
        
    except Exception as e:
        failed_indices.append(idx)
        print(f"[{i}/{len(Watchlist)}] ✗ {idx} - Error: {e}")

# Combine and save
if all_data:
    df_combined = pd.concat(all_data, ignore_index=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'nse_indices_{timestamp}.csv'
    df_combined.to_csv(filename, index=False)
    print(f"\n✓ Success! Saved {len(all_data)} indices to {filename}")
    print(f"  Total rows: {len(df_combined)}")
else:
    print("\n✗ No data fetched!")

if failed_indices:
    print(f"\n⚠ Failed to fetch {len(failed_indices)} indices:")
    for idx in failed_indices:
        print(f"  - {idx}")

Fetching data for 32 indices...

[1/32] ✗ Nifty India Consumption - Error: list indices must be integers or slices, not str
[2/32] ✗ Nifty Financial Services - Error: list indices must be integers or slices, not str
[3/32] ✗ Nifty Pharma - Error: list indices must be integers or slices, not str
[4/32] ✗ Nifty Auto - Error: list indices must be integers or slices, not str
[5/32] ✗ Nifty Private Bank - Error: list indices must be integers or slices, not str
[6/32] ✗ Nifty Commodities - Error: list indices must be integers or slices, not str
[7/32] ✗ Nifty MNC - Error: list indices must be integers or slices, not str
[8/32] ✗ Nifty Bank - Error: list indices must be integers or slices, not str
[9/32] ✗ Nifty Metal - Error: list indices must be integers or slices, not str
[10/32] ✗ Nifty Services Sector - Error: list indices must be integers or slices, not str
[11/32] ✗ Nifty PSE - Error: list indices must be integers or slices, not str
[12/32] ✗ Nifty Infrastructure - Error: list indices 

In [15]:
watchlist = [
    "NIFTY INDIA CONSUMPTION",
    "NIFTY FINANCIAL SERVICES",
    "NIFTY PHARMA",
    "NIFTY AUTO",
    "NIFTY PRIVATE BANK",
    "NIFTY COMMODITIES",
    "NIFTY MNC",
    "NIFTY BANK",
    "NIFTY METAL",
    "NIFTY SERVICES SECTOR",
    "NIFTY PSE",
    "NIFTY INFRASTRUCTURE",
    "NIFTY SMALLCAP 100",
    "NIFTY TATA GROUP 25% CAP",
    "NIFTY MIDCAP 50",
    "NIFTY PSU BANK",
    "NIFTY ENERGY",
    "NIFTY IT",
    "NIFTY FMCG",
    "NIFTY CPSE",
    "NIFTY50 USD",
    "NIFTY OIL & GAS",
    "NIFTY NEXT 50",
    "NIFTY 50",
    "NIFTY GROWTH SECTORS 15",
    "NIFTY 100",
    "NIFTY 500",
    "NIFTY MIDCAP 100",
    "NIFTY FINANCIAL SERVICES 25/50",
    "NIFTY REALTY",
    "NIFTY HEALTHCARE INDEX",
    "NIFTY CONSUMER DURABLES"
]

# Get stock lists
all_stocks = []

print(f"Fetching stock lists for {len(watchlist)} indices...\n")

for i, idx in enumerate(watchlist, 1):
    try:
        # Get list of symbols
        symbols = c.get_index_details(idx, list_only=True)

        if symbols:
            for symbol in symbols:
                all_stocks.append({
                    'index_name': idx,
                    'symbol': symbol
                })
            print(f"[{i}/{len(watchlist)}] ✓ {idx} ({len(symbols)} stocks)")
        else:
            print(f"[{i}/{len(watchlist)}] ✗ {idx} - No stocks found")

    except Exception as e:
        print(f"[{i}/{len(watchlist)}] ✗ {idx} - Error: {e}")

# Convert to DataFrame and save
if all_stocks:
    df = pd.DataFrame(all_stocks)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'nse_stock_lists_{timestamp}.csv'

    df.to_csv(filename, index=False)

    print(f"\n{'='*60}")
    print(f"✓ SUCCESS!")
    print(f"  File: {filename}")
    print(f"  Total entries: {len(df)}")
    print(f"  Unique stocks: {df['symbol'].nunique()}")
    print(f"{'='*60}")
else:
    print("\n✗ No data collected!")

Fetching stock lists for 32 indices...

[1/32] ✓ NIFTY INDIA CONSUMPTION (31 stocks)
[2/32] ✓ NIFTY FINANCIAL SERVICES (21 stocks)
[3/32] ✓ NIFTY PHARMA (21 stocks)
[4/32] ✓ NIFTY AUTO (16 stocks)
[5/32] ✓ NIFTY PRIVATE BANK (11 stocks)
[6/32] ✓ NIFTY COMMODITIES (31 stocks)
[7/32] ✓ NIFTY MNC (31 stocks)
[8/32] ✓ NIFTY BANK (13 stocks)
[9/32] ✓ NIFTY METAL (16 stocks)
[10/32] ✓ NIFTY SERVICES SECTOR (31 stocks)
[11/32] ✓ NIFTY PSE (21 stocks)
[12/32] ✓ NIFTY INFRASTRUCTURE (31 stocks)
[13/32] ✓ NIFTY SMALLCAP 100 (101 stocks)
[14/32] ✗ NIFTY TATA GROUP 25% CAP - No stocks found
[15/32] ✓ NIFTY MIDCAP 50 (51 stocks)
[16/32] ✓ NIFTY PSU BANK (13 stocks)
[17/32] ✓ NIFTY ENERGY (41 stocks)
[18/32] ✓ NIFTY IT (11 stocks)
[19/32] ✓ NIFTY FMCG (16 stocks)
[20/32] ✓ NIFTY CPSE (12 stocks)
[21/32] ✗ NIFTY50 USD - No stocks found
[22/32] ✓ NIFTY OIL & GAS (16 stocks)
[23/32] ✓ NIFTY NEXT 50 (51 stocks)
[24/32] ✓ NIFTY 50 (51 stocks)
[25/32] ✓ NIFTY GROWTH SECTORS 15 (16 stocks)
[26/32] ✓ NIFTY 

In [16]:
u = c.get_fno_full_list()

In [18]:
u.Symbol

0          360ONE
1             ABB
2       APLAPOLLO
3          AUBANK
4      ADANIENSOL
5        ADANIENT
6      ADANIGREEN
7      ADANIPORTS
8       ABCAPITAL
9           ALKEM
10          AMBER
11      AMBUJACEM
12       ANGELONE
13     APOLLOHOSP
14       ASHOKLEY
15     ASIANPAINT
16         ASTRAL
17     AUROPHARMA
18          DMART
19       AXISBANK
20            BSE
21     BAJAJ-AUTO
22     BAJFINANCE
23     BAJAJFINSV
24     BANDHANBNK
25     BANKBARODA
26      BANKINDIA
27            BDL
28            BEL
29     BHARATFORG
30           BHEL
31           BPCL
32     BHARTIARTL
33         BIOCON
34     BLUESTARCO
35       BOSCHLTD
36      BRITANNIA
37        CGPOWER
38          CANBK
39           CDSL
40       CHOLAFIN
41          CIPLA
42      COALINDIA
43        COFORGE
44         COLPAL
45           CAMS
46         CONCOR
47       CROMPTON
48     CUMMINSIND
49         CYIENT
50            DLF
51          DABUR
52      DALBHARAT
53      DELHIVERY
54       DIVISLAB
55        